<a href="https://colab.research.google.com/github/inthecng/SmartThingsPublic/blob/master/naver_land.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import requests
import pandas as pd


# 데이터 크롤링

# API 기본 URL 설정
url = "https://m.land.naver.com/cluster/ajax/articleList"

# 요청 헤더 설정 (웹 브라우저에서 보낸 요청처럼 보이도록 설정)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# 페이지 크기와 초기 페이지 번호 설정
page_size = 20  # 한 페이지당 항목 수
page_number = 1  # 시작 페이지 번호

# 모든 데이터를 담을 리스트
all_data = []

# 데이터가 없을 때까지 반복해서 API 호출
while True:
    # URL에 페이지와 페이지 크기 추가 (params는 쿼리 파라미터)
    params = {
        "rletTpCd": "DDDGG",  # 거래 유형 코드
        "tradTpCd": "B2",  # 거래 타입 코드
        "z": 14,  # 줌 레벨
        "lat": 37.0535034,  # 위도
        "lon": 127.0258068,  # 경도
        "btm": 37.0237004,  # 남쪽 경계
        "lft": 126.9792436,  # 왼쪽 경계
        "top": 37.0832946,  # 북쪽 경계
        "rgt": 127.0723699,  # 오른쪽 경계
        "showR0": "",  # (빈 값으로 설정)
        "page": page_number,  # 현재 페이지
        "size": page_size  # 페이지 크기
    }

    # API 호출 (requests 라이브러리 사용)
    response = requests.get(url, headers=headers, params=params, verify=False)

    # 응답이 정상적이면 (상태 코드 200)
    if response.status_code == 200:
        data = response.json()  # JSON 응답 파싱

        # 데이터가 없으면 종료
        if not data.get('body'):
            break

        # 응답 데이터 추가 (리스트에 데이터 누적)
        all_data.extend(data['body'])

        # 다음 페이지로 이동
        page_number += 1
    else:
        print(f"API 호출 실패: {response.status_code}")
        break  # API 호출 실패 시 종료

# 수집된 데이터를 pandas DataFrame으로 변환
df_org = pd.DataFrame(all_data)
df = df_org.copy()


################################################################################
# Google Sheets와 연동을 위한 코드
################################################################################

import gspread
from google.oauth2.service_account import Credentials

# 인증 범위 설정 (Google Sheets API 사용을 위한 범위)
scopes = ["https://www.googleapis.com/auth/spreadsheets"]

# 서비스 계정 인증서 로드 (Google Cloud 서비스 계정 JSON 파일 사용)
credentials = Credentials.from_service_account_file(
    'naverland-446111-584ea20bf298.json', scopes=scopes)

# gspread 클라이언트 생성 (구글 스프레드시트와의 연결)
gc = gspread.authorize(credentials)

# 스프레드시트 URL로 열기
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Eid96Vuvj-2FLDfVLxV98QUzAo0GbuItxqZv0UbxJT4/edit?usp=sharing'
spreadsheet = gc.open_by_url(spreadsheet_url)

# 'column_code' 워크시트 선택
worksheet = spreadsheet.worksheet('column_code')

# 워크시트의 모든 데이터를 가져와 DataFrame으로 변환
data = worksheet.get_all_records()
code_df = pd.DataFrame(data)

# 'Code'와 '구분2' 컬럼을 기준으로 딕셔너리 생성
mapping_dict = dict(zip(code_df['Code'], code_df['구분2']))

# 기존 DataFrame(df)의 컬럼을 딕셔너리를 이용해 변경
df = df.rename(columns=mapping_dict)
df = df_org_1.copy()



,atclNo,cortarNo,단지명,atclStatCd,rletTpCd,uprRletTpCd,매물유형,tradTpCd,거래구분,vrfcTpCd,...,cpLinkVO,dtlAddrYn,dtlAddr,isVrExposed,vrUrl,sameAddrHash,sameAddrMaxPrc,sameAddrMaxPrc2,sameAddrMinPrc,sameAddrMinPrc2
0,2461860210,4122032000,다가구,R0,C03,C03,단독/다가구,B2,월세,S_VR,...,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,True,https://fin.land.naver.comnull,NaN,NaN,NaN,NaN,NaN
1,2501250222,4122010100,다가구,R0,C03,C03,단독/다가구,B2,월세,OWNER,...,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2501254718,4122033000,단독,R0,C03,C03,단독/다가구,B2,월세,NDOC1,...,"{'cpId': 'industry', 'mobileArticleUrl': 'http...",N,,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2501231015,4122012800,다가구,R0,C03,C03,단독/다가구,B2,월세,NDOC1,...,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,False,NaN,22C03B2N491b2da544f3d8807b4f664188bdd2e04d17f1...,500,79,500,75
4,2501235181,4122012800,다가구,R0,C03,C03,단독/다가구,B2,월세,OWNER,...,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,2462177069,4122012800,다가구,R0,C03,C03,단독/다가구,B2,월세,NONE,...,"{'cpId': 'KAR', 'mobileArticleLinkTypeCode': '...",N,,False,NaN,18C03B2Y255880335dba95a2900df3936339e2e351259a...,"1,000",120,500,70
598,2462053511,4122032000,다가구,R0,C03,C03,단독/다가구,B2,월세,NONE,...,"{'cpId': 'KAR', 'mobileArticleLinkTypeCode': '...",N,,False,NaN,NaN,NaN,NaN,NaN,NaN
599,2461653079,4122010100,다가구,R0,C03,C03,단독/다가구,B2,월세,NONE,...,"{'cpId': 'KAR', 'mobileArticleLinkTypeCode': '...",N,,False,NaN,NaN,NaN,NaN,NaN,NaN
600,2461628592,4122010100,다가구,R0,C03,C03,단독/다가구,B2,월세,NONE,...,"{'cpId': 'KAR', 'mobileArticleLinkTypeCode': '...",N,,False,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df = df.head(5)

In [21]:
################################################################################
# 도로명 주소 붙이기
################################################################################

# 네이버 API 키 (본인의 API 키로 바꾸세요)
client_id = '8glqus7zgt'  # API Key ID
client_secret = 'hVBVbCX63Y1klYxjwTuGrlqSTLRgH27nWS0Sh8B3'  # API Key

# 네이버 API 호출 함수
def get_address_from_coords(위도, 경도):
    try:
        # 요청 URL 생성
        coords = f"{경도},{위도}"
        url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={coords}&output=json&orders=roadaddr"

        # 헤더에 API 키 추가
        headers = {
            'x-ncp-apigw-api-key-id': client_id,
            'x-ncp-apigw-api-key': client_secret
        }

        # API 호출
        response = requests.get(url, headers=headers, verify=False)

        # 응답을 JSON 형식으로 파싱
        data = response.json()

        # 주소 정보를 추출하여 반환
        if data['status']['code'] == 0:
            address_data = {
                '도': data['results'][0]['region']['area1']['name'],
                '시': data['results'][0]['region']['area2']['name'],
                '동': data['results'][0]['region']['area3']['name'],
                '도로명': data['results'][0]['land']['name'],
                '번호1': data['results'][0]['land']['number1'],
                '번호2': data['results'][0]['land']['number2']
            }
            return address_data
        else:
            return {'경기도': None, '평택시': None, '고덕동': None, '함박산5길': None, '번호': None}
    except Exception as e:
        print(f"Error: {e}")
        return {'경기도': None, '평택시': None, '고덕동': None, '함박산5길': None, '번호': None}

# '위도'와 '경도' 컬럼을 기준으로 주소를 가져와서 새로운 컬럼에 추가
addresses = []
for index, row in df.iterrows():
    address_data = get_address_from_coords(row['위도'], row['경도'])
    addresses.append(address_data)

# 각 주소 데이터를 데이터프레임으로 변환하여 추가
address_df = pd.DataFrame(addresses)

# 원본 df와 주소 데이터를 합침
df = pd.concat([df, address_df], axis=1)

################################################################################
# 불필요한 자료 정리
################################################################################

# 고덕동 자료만 필터링
df = df[df['동'].isin(['고덕동'])]


ValueError: cannot reindex on an axis with duplicate labels

In [23]:
df[df['동'].isin(['고덕동'])]


ValueError: cannot reindex on an axis with duplicate labels

In [22]:
df

,atclNo,cortarNo,단지명,atclStatCd,rletTpCd,uprRletTpCd,매물유형,tradTpCd,거래구분,vrfcTpCd,...,동,도로명,번호1,번호2,도,시,동,도로명,번호1,번호2
0,2461860210,4122032000,다가구,R0,C03,C03,단독/다가구,B2,월세,S_VR,...,서탄면,장등안길,29,9,경기도,평택시,서탄면,장등안길,29,9
1,2501250222,4122010100,다가구,R0,C03,C03,단독/다가구,B2,월세,OWNER,...,서정동,탄현로,180,,경기도,평택시,서정동,탄현로,180,
2,2501254718,4122033000,단독,R0,C03,C03,단독/다가구,B2,월세,NDOC1,...,고덕동,고덕갈평8길,20,,경기도,평택시,고덕동,고덕갈평8길,20,
3,2501231015,4122012800,다가구,R0,C03,C03,단독/다가구,B2,월세,NDOC1,...,고덕동,고덕갈평10길,53,7,경기도,평택시,고덕동,고덕갈평10길,53,7
4,2501235181,4122012800,다가구,R0,C03,C03,단독/다가구,B2,월세,OWNER,...,고덕동,고덕여염17길,119,,경기도,평택시,고덕동,고덕여염17길,119,
